In [3]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import StrainerStatus
from benchmark_consts import *

In [4]:
qs = UNSTABLE_VERUS
options = DebugOptions()
options.is_verus = True

mv = BenchViewer(qs, options)

[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] 8d058577cd trace slow unknown, fallback to timeout 


In [5]:
mv.status.print_status()

| category                   |   count | percentage   |
|----------------------------|---------|--------------|
| StrainerStatus.FINISHED    |      65 | 92.86 %      |
| StrainerStatus.NOT_CREATED |       3 | 4.29 %       |
| StrainerStatus.NO_PROOF    |       2 | 2.86 %       |
| total                      |      70 | 100.00 %     |


In [6]:
discount = [
    ("08719204aa", "4b9e624e"),
    ("81b159a977", "fbe192f3"),
    ("2556504d15", "7ab783b0"),
    ("6540edb885", "5e546c11"),
    ("82449bd2e8", "4c612f9b"),
    ("a1b4b3fd4a", "4e444e51"),
    ("55b4af82a4", "02d50371"),
    ("2556504d15", "0e424555"),
    ("4d2d4fbc20", "5a3b4208"),
    ("82449bd2e8", "905f3174"),
    ("2045867a58", "7ec12ea5"),
    ("55b4af82a4", "781157b6"),
    ("68ac429152", "75d801e3"),
]

In [7]:
def fixable_after_discount(dbg):
    report =dbg.report
    name_hash = dbg.name_hash
    fixable = False
    for edit_path in report.stabilized['edit_path']:
        edit_hash = os.path.basename(edit_path).split('.')[0]
        if (name_hash, edit_hash) in discount:
            continue
        fixable = True
    return fixable

In [8]:
from utils.analysis_utils import fmt_percent

bench_total = len(mv.status.tally)
bench_finished = len(mv.status[StrainerStatus.FINISHED])
# to_fixable = 0
to_cases = dict()
ff_cases = dict()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

for q in mv.status[StrainerStatus.FINISHED]:
    dbg = mv[q]
    if dbg.mode == DbgMode.TIMEOUT:
        to_cases[q] = q in mv.fixable and fixable_after_discount(dbg)
    elif dbg.mode == DbgMode.FAST_FAIL:
        ff_cases[q] = q in mv.fixable and fixable_after_discount(dbg)
    else:
        print("Unexpected mode", dbg.mode)
        assert False

to_fixable = sum(to_cases.values())
ff_fixable = sum(ff_cases.values())

print("Test Finished:", print_break_down(bench_finished, bench_total))
print("\tTOs:", print_break_down(len(to_cases), bench_finished))
print("\t\tTO fixable:", print_break_down(to_fixable, len(to_cases)))
print("\tFFs:", print_break_down(len(ff_cases), bench_finished))
print("\t\tFF fixable:", print_break_down(ff_fixable, len(ff_cases)))

print("Test Not Finished:", print_break_down(len(mv.status[StrainerStatus.NOT_CREATED]), bench_total))
print("No Proofs:", print_break_down(len(mv.status[StrainerStatus.NO_PROOF]), bench_total))
print("* fixable:", print_break_down(to_fixable + ff_fixable, bench_total))

Test Finished: 65/70 (92.9%)
	TOs: 38/65 (58.5%)
		TO fixable: 25/38 (65.8%)
	FFs: 27/65 (41.5%)
		FF fixable: 22/27 (81.5%)
Test Not Finished: 3/70 (4.3%)
No Proofs: 2/70 (2.9%)
* fixable: 47/70 (67.1%)


In [9]:
for query in mv.fixable:
    report = mv[query].report
    name_hash = mv[query].name_hash

    for edit_path in report.stabilized['edit_path']:
        edit_query = os.path.basename(edit_path)
        bundle_path = f"data/projs/verus_singleton_bundle/base.z3/{name_hash}.{edit_query}"

In [10]:
for q in mv.status[StrainerStatus.NO_PROOF]:
    print(q)

data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg__impl_%4__merge_with_before_ll_inv_valid_unused.smt2
data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.4.smt2


In [11]:
for q in mv.status[StrainerStatus.NOT_CREATED]:
    print("./src/debugger3.py -m fast_fail -i", mv[q].name_hash, "--create-project")

./src/debugger3.py -m fast_fail -i d6181053ff --create-project
./src/debugger3.py -m fast_fail -i 47ae6b0135 --create-project
./src/debugger3.py -m fast_fail -i d057fff3f5 --create-project


In [12]:
from utils.system_utils import list_smt2_files

for q in mv.status[StrainerStatus.UNFILTERED]:
    assert len(list_smt2_files(mv[q].strainer.filter_dir)) != 0
    print("./src/make_spaghet.py --verus -i", mv[q].strainer.filter_dir)

for q in mv.status[StrainerStatus.NOT_TESTED]:
    assert len(list_smt2_files(mv[q].strainer.test_dir)) != 0
    print("./src/make_spaghet.py --verus -i", mv[q].strainer.test_dir)

In [18]:
from utils.cluster_utils import get_sync_commands
from utils.system_utils import write_misc_script

for q in mv.status.tally:
    dbg = mv[q]
    dbg.collect_garbage()

write_misc_script("send.sh", get_sync_commands("g2001", mv.get_sync_dirs()))

[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_CREATED data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.3.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_CREATED data/projs/verismo.dedup/base.z3/arch__ptram__ptram_p2.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg__impl_%4__merge_with_before_ll_inv_valid_unused.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_CREATED data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.5.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.4.smt2 
